In [ ]:
# @title (Run me, no need to look) Let's use this local copy of the labellines module (because it is small and speeds up our setup)
import warnings
from typing import List, Optional, Union
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.container import ErrorbarContainer
from matplotlib.dates import DateConverter, num2date
from matplotlib.lines import Line2D
from more_itertools import always_iterable
from __future__ import annotations
from typing import TYPE_CHECKING
import matplotlib.patheffects as patheffects
import numpy as np
from matplotlib.text import Text
from typing import Tuple



def normalize_xydata(line: Line2D) -> Tuple[np.ndarray, np.ndarray]:
    """Make sure datetime values are properly converted to floats and convert
    into data coordinates."""
    # Convert the data into the data coordinates
    line_t = line.get_transform().transform
    axes_t = line.axes.transData.inverted().transform
    x, y = axes_t(line_t(line.get_xydata())).T

    return x, y


# From https://www.geeksforgeeks.org/maximum-bipartite-matching/
class GFG:
    def __init__(self, graph):
        # residual graph
        self.graph = graph
        self.ppl = len(graph)
        self.jobs = len(graph[0])

    # A DFS based recursive function that returns true if a matching for vertex
    # u is possible
    def bpm(self, u, match_r, seen):
        # Try every job one by one
        for v in range(self.jobs):
            # If applicant u is interested
            # in job v and v is not seen
            if self.graph[u][v] and not seen[v]:
                # Mark v as visited
                seen[v] = True

                # If job 'v' is not assigned to an applicant OR previously
                # assigned applicant for job v (which is matchR[v]) has an
                # alternate job available. Since v is marked as visited in the
                # above line, matchR[v] in the following recursive call will not
                # get job 'v' again

                if match_r[v] == -1 or self.bpm(match_r[v], match_r, seen):
                    match_r[v] = u
                    return True
        return False

    # Returns maximum number of matching
    def maxBPM(self):
        # An array to keep track of the applicants assigned to jobs. The value
        # of matchR[i] is the applicant number assigned to job i, the value -1
        # indicates nobody is assigned.
        matchR = [-1] * self.jobs

        # Count of jobs assigned to applicants
        result = 0
        for i in range(self.ppl):
            # Mark all jobs as not seen for next applicant.
            seen = [False] * self.jobs

            # Find if the applicant 'u' can get a job
            if self.bpm(i, matchR, seen):
                result += 1
        return result, matchR


def maximum_bipartite_matching(graph: np.ndarray) -> np.ndarray:
    """Finds the maximum bipartite matching of a graph

    Parameters
    ----------
    graph : np.ndarray
        The graph, represented as a boolean matrix

    Returns
    -------
    order : np.ndarray
        The order in which to traverse the graph to visit a maximum of nodes
    """
    g = GFG(graph)
    _, order = g.maxBPM()
    return np.asarray(order)



if TYPE_CHECKING:
    from datetime import datetime
    from typing import Any, Literal, Optional, Union

    from matplotlib.axes import Axes
    from matplotlib.lines import Line2D

    Position = Union[float, datetime, np.datetime64]
    ColorLike = Any  # mpl has no type annotations so this is just a crutch
    AutoLiteral = Literal["auto"]


class LineLabel(Text):
    """This artist adds a label onto a preexisting Line2D object"""

    _line: Line2D
    """Annotated line"""

    _target_x: Position
    """Requested x position of the label, as supplied by the user"""

    _ax: Axes
    """Axes containing the line"""

    _auto_align: bool
    """Align text with the line (True) or parallel to x axis (False)"""

    _yoffset: float
    """An additional y offset for the label"""

    _yoffset_logspace: bool
    """Sets whether to treat _yoffset exponentially"""

    _label_pos: np.ndarray
    """Position of the label, computed from _target_x and line data"""

    _anchor_a: np.ndarray
    """Anchor A for rotation calculation, point of _line neighbouring this label"""

    _anchor_b: np.ndarray
    """Anchor B for rotation calculation, point of _line neighbouring this label"""

    def __init__(
        self,
        line: Line2D,
        x: Position,
        label: Optional[str] = None,
        align: Optional[bool] = None,
        yoffset: float = 0,
        yoffset_logspace: bool = False,
        outline_color: Optional[Union[AutoLiteral, ColorLike]] = "auto",
        outline_width: float = 8,
        rotation: Optional[float] = None,
        **kwargs,
    ) -> None:
        """

        Parameters
        ----------
        line : Line2D
            Line to be decorated.
        x : Position
            Horizontal target position for the label (in data units).
        label : str, optional
            Override for line label, by default None.
        align : bool, optional
            If true, the label is parallel to the line, otherwise horizontal,
            by default True.
        yoffset : float, optional
            An additional y offset for the line label, by default 0.
        yoffset_logspace : bool, optional
            If true yoffset is applied exponentially to appear linear on a log-axis,
            by default False.
        outline_color : None | "auto" | Colorlike
            Colour of the outline. If set to "auto", use the background color.
            If set to None, do not draw an outline, by default "auto".
        outline_width : float
            Width of the outline, by default 8.
        rotation: float, optional
            If set and align = False, controls the angle of the label.
        """
        # When rotation is set, align has to be false or None
        if rotation is not None and align:
            raise ValueError(
                f"When rotation is set, align needs to be false or none was {align=}."
            )
        elif rotation is None:
            align = True if (align or align is None) else False
            rotation = 0
        elif rotation is None and not align:
            align = False
            rotation = 0
        elif not align or align is None:
            align = False
            rotation = rotation

        self._line = line
        self._target_x = x
        self._ax = line.axes
        self._auto_align = align
        self._yoffset = yoffset
        self._yoffset_logspace = yoffset_logspace
        label = label or line.get_label()

        # Populate self._pos, self._anchor_a, self._anchor_b
        self._update_anchors()
        self._rotation = rotation

        # Set a bunch of default arguments
        kwargs.setdefault("color", self._line.get_color())
        kwargs.setdefault("clip_on", True)
        kwargs.setdefault("zorder", 2.5)
        if "ha" not in kwargs:
            kwargs.setdefault("horizontalalignment", "center")
        if "va" not in kwargs:
            kwargs.setdefault("verticalalignment", "center")

        # Initialize Text Artist
        super().__init__(
            *self._label_pos,
            label,
            rotation=self._rotation,
            rotation_mode="anchor",
            **kwargs,
        )

        # Apply outline effect
        if outline_color is not None:
            if outline_color == "auto":
                outline_color = line.axes.get_facecolor()

            self.set_path_effects(
                [
                    patheffects.Stroke(
                        linewidth=outline_width, foreground=outline_color
                    ),
                    patheffects.Normal(),
                ]
            )

        # activate clipping if needed and place on axes
        if kwargs["clip_on"]:
            self.set_clip_path(self._ax.patch)
        self._ax._add_text(self)

    def _update_anchors(self):
        """
        This helper method computes the position of the textbox and determines
        the anchor points needed to adjust the rotation
        """
        # Use the mpl-internal float representation (deals with datetime etc)
        x = self._line.convert_xunits(self._target_x)
        xdata, ydata = normalize_xydata(self._line)

        mask = np.isfinite(ydata)
        if mask.sum() == 0:
            raise ValueError(f"The line {self._line} only contains nan!")

        # Find the first line segment surrounding x
        for i, (xa, xb) in enumerate(zip(xdata[:-1], xdata[1:])):
            if min(xa, xb) <= x <= max(xa, xb):
                ya, yb = ydata[i], ydata[i + 1]
                break
        else:
            raise ValueError("x label location is outside data range!")

        # Interpolate y position of label, (interp needs sorted data)
        if xa != xb:
            dx = np.array((xa, xb))
            dy = np.array((ya, yb))
            srt = np.argsort(dx)
            y = np.interp(x, dx[srt], dy[srt])
        else:  # Vertical case
            y = (ya + yb) / 2

        # Apply y offset
        if self._yoffset_logspace:
            y *= 10**self._yoffset
        else:
            y += self._yoffset

        if not np.isfinite(y):
            raise ValueError(
                f"{self._line} does not have a well defined value"
                f" at x = {self._target_x}. Consider a different position."
            )

        self._label_pos = np.array((x, y))
        self._anchor_a = np.array((xa, ya))
        self._anchor_b = np.array((xb, yb))

    def __auto_align(self, value=None):
        # Helper function  to help  resize the  alignment of
        # the label if the window is resized
        # Providing the _rotation property
        # enables automatic adjustment of the rotation angle
        # Adapted from https://stackoverflow.com/a/53111799
        if self._auto_align:
            # Transform to screen coordinated to make sure the angle is always
            # correct regardless of axis scaling etc.
            xa, ya = self._ax.transData.transform(self._anchor_a)
            xb, yb = self._ax.transData.transform(self._anchor_b)
            angle = np.rad2deg(np.arctan2(yb - ya, xb - xa))

            # Correct the angle to make sure text is always upright-ish
            value = (angle + 90) % 180 - 90
        if isinstance(value, (float, int)):
            self.__rotation = value
        return self.__rotation

    @property
    def _rotation(self):
        return self.__auto_align()

    @_rotation.setter
    def _rotation(self, rotation):
        self.__rotation = self.__auto_align(rotation)

# Label line with line2D label data
def labelLine(
    line: Line2D,
    x,
    label: Optional[str] = None,
    align: Optional[bool] = None,
    drop_label: bool = False,
    yoffset: float = 0,
    yoffset_logspace: bool = False,
    outline_color: str = "auto",
    outline_width: float = 8,
    rotation: Optional[float] = None,
    **kwargs,
):
    """
    Label a single matplotlib line at position x

    Parameters
    ----------
    line : matplotlib.lines.Line
       The line holding the label
    x : number
       The location in data unit of the label
    label : string, optional
       The label to set. This is inferred from the line by default
    align : boolean, optional
       If True, the label will be aligned with the slope of the line
       at the location of the label. If False, they will be horizontal.
    drop_label : bool, optional
       If True, the label is consumed by the function so that subsequent
       calls to e.g. legend do not use it anymore.
    yoffset : double, optional
        Space to add to label's y position
    yoffset_logspace : bool, optional
        If True, then yoffset will be added to the label's y position in
        log10 space
    outline_color : None | "auto" | color
        Colour of the outline. If set to "auto", use the background color.
        If set to None, do not draw an outline.
    outline_width : number
        Width of the outline
    rotation: float, optional
            If set and align = False, controls the angle of the label
    kwargs : dict, optional
       Optional arguments passed to ax.text
    """

    try:
        txt = LineLabel(
            line,
            x,
            label=label,
            align=align,
            yoffset=yoffset,
            yoffset_logspace=yoffset_logspace,
            outline_color=outline_color,
            outline_width=outline_width,
            rotation=rotation,
            **kwargs,
        )
    except ValueError as err:
        if "does not have a well defined value" in str(err):
            warnings.warn(
                (
                    "%s could not be annotated due to `nans` values. "
                    "Consider using another location via the `x` argument."
                )
                % line,
                UserWarning,
                stacklevel=1,
            )
            return
        raise err

    if drop_label:
        line.set_label(None)

    return txt


def labelLines(
    lines: Optional[List[Line2D]] = None,
    align: Optional[bool] = None,
    xvals: Optional[Union[tuple[float, float], list[float]]] = None,
    drop_label: bool = False,
    shrink_factor: float = 0.05,
    yoffsets: Union[float, list[float]] = 0,
    outline_color: str = "auto",
    outline_width: float = 5,
    rotation: Optional[bool] = None,
    **kwargs,
):
    """Label all lines with their respective legends.

    Parameters
    ----------
    lines : list of matplotlib lines, optional.
       Lines to label. If empty, label all lines that have a label.
    align : boolean, optional
       If True, the label will be aligned with the slope of the line
       at the location of the label. If False, they will be horizontal.
    xvals : (xfirst, xlast) or array of float, optional
       The location of the labels. If a tuple, the labels will be
       evenly spaced between xfirst and xlast (in the axis units).
    drop_label : bool, optional
       If True, the label is consumed by the function so that subsequent
       calls to e.g. legend do not use it anymore.
    shrink_factor : double, optional
       Relative distance from the edges to place closest labels. Defaults to 0.05.
    yoffsets : number or list, optional.
        Distance relative to the line when positioning the labels. If given a number,
        the same value is used for all lines.
    outline_color : None | "auto" | color
        Colour of the outline. If set to "auto", use the background color.
        If set to None, do not draw an outline.
    outline_width : number
        Width of the outline
    rotation: float, optional
        If set and align = False, controls the angle of the label
    kwargs : dict, optional
       Optional arguments passed to ax.text
    """
    if lines:
        ax = lines[0].axes
    else:
        ax = plt.gca()

    handles, labels_of_handles = ax.get_legend_handles_labels()

    all_lines, all_labels = [], []
    for h, label in zip(handles, labels_of_handles):
        if isinstance(h, ErrorbarContainer):
            line = h.lines[0]
        else:
            line = h

        # If the user provided a list of lines to label, only label those
        if (lines is not None) and (line not in lines):
            continue
        all_lines.append(line)
        all_labels.append(label)

    # Check that the lines passed to the function have all a label
    if lines is not None:
        for line in lines:
            if line in all_lines:
                continue

            warnings.warn(
                "Tried to label line %s, but could not find a label for it." % line,
                UserWarning,
                stacklevel=1,
            )

    # In case no x location was provided, we need to use some heuristics
    # to generate them.
    if xvals is None:
        xvals = ax.get_xlim()
        xscale = ax.get_xscale()
        if xscale == "log":
            log10_xvals = np.log10(xvals)
            xvals_rng = log10_xvals[1] - log10_xvals[0]
            shrinkage = xvals_rng * shrink_factor
            xvals = (
                10 ** (log10_xvals[0] + shrinkage),
                10 ** (log10_xvals[1] - shrinkage),
            )
        else:
            xvals_rng = xvals[1] - xvals[0]
            shrinkage = xvals_rng * shrink_factor
            xvals = (xvals[0] + shrinkage, xvals[1] - shrinkage)

    if isinstance(xvals, tuple) and len(xvals) == 2:
        xmin, xmax = xvals
        xscale = ax.get_xscale()
        if xscale == "log":
            xvals = np.geomspace(xmin, xmax, len(all_lines) + 2)[1:-1]
        else:
            xvals = np.linspace(xmin, xmax, len(all_lines) + 2)[1:-1]

        # Build matrix line -> xvalue
        ok_matrix = np.zeros((len(all_lines), len(all_lines)), dtype=bool)

        for i, line in enumerate(all_lines):
            xdata, _ = normalize_xydata(line)
            minx, maxx = min(xdata), max(xdata)
            for j, xv in enumerate(xvals):  # type: ignore
                ok_matrix[i, j] = minx < xv < maxx

        # If some xvals do not fall in their corresponding line,
        # find a better matching using maximum bipartite matching.
        if not np.all(np.diag(ok_matrix)):
            order = maximum_bipartite_matching(ok_matrix)

            # The maximum match may miss a few points, let's add them back
            order[order < 0] = np.setdiff1d(np.arange(len(order)), order[order >= 0])

            # Now reorder the xvalues
            old_xvals = xvals.copy()  # type: ignore
            xvals[order] = old_xvals  # type: ignore
    else:
        xvals = list(always_iterable(xvals))  # force the creation of a copy

    lab_lines, labels = [], []
    # Take only the lines which have labels other than the default ones
    for i, (line, xv) in enumerate(zip(all_lines, xvals)):  # type: ignore
        label = all_labels[all_lines.index(line)]
        lab_lines.append(line)
        labels.append(label)

        # Move xlabel if it is outside valid range
        xdata, _ = normalize_xydata(line)
        if not (min(xdata) <= xv <= max(xdata)):
            warnings.warn(
                (
                    "The value at position {} in `xvals` is outside the range of its "
                    "associated line (xmin={}, xmax={}, xval={}). Clipping it "
                    "into the allowed range."
                ).format(i, min(xdata), max(xdata), xv),
                UserWarning,
                stacklevel=1,
            )
            new_xv = min(xdata) + (max(xdata) - min(xdata)) * 0.9
            xvals[i] = new_xv  # type: ignore

    # Convert float values back to datetime in case of datetime axis
    if isinstance(ax.xaxis.converter, DateConverter):
        xvals = [
            num2date(x).replace(tzinfo=ax.xaxis.get_units())
            for x in xvals  # type: ignore
        ]

    txts = []
    try:
        yoffsets = [float(yoffsets)] * len(all_lines)  # type: ignore
    except TypeError:
        pass
    for line, x, yoffset, label in zip(
        lab_lines, xvals, yoffsets, labels  # type: ignore
    ):
        txts.append(
            labelLine(
                line,
                x,
                label=label,
                align=align,
                drop_label=drop_label,
                yoffset=yoffset,
                outline_color=outline_color,
                outline_width=outline_width,
                rotation=rotation,
                **kwargs,
            )
        )

    return txts

In [ ]:
# @title (read only) This cell updates the Truss class to allow us to simulate forces on our bridge


import sympy
from sympy.physics.continuum_mechanics.truss import Truss
import matplotlib.pyplot as plt
import matplotlib.colors as mcm
from matplotlib.patches import PathPatch
from matplotlib.path import Path
import numpy as np
import os
import copy
import pickle
import time as time_mod
from scipy.interpolate import interp1d
import pandas as pd
import time as time_module


ff = os.path.join(os.getcwd(), 'frames')
if not os.path.isdir(ff):
    os.mkdir(ff)


class Truss2(Truss):

    def __init__(self):
        """
        Initializes the class
        """
        self._nodes = []
        self._members = {}
        self._loads = {}
        self._supports = {}
        self._node_labels = []
        self._node_positions = []
        self._node_position_x = []
        self._node_position_y = []
        self._nodes_occupied = {}
        self._reaction_loads = {}
        self._internal_forces = {}
        self._node_coordinates = {}
        self._history = {'time': [], 'max_comp': [], 'max_tens': [],'max_comp_id': [], 'max_tens_id': []}
        self._sf = 3e3

    def get_member_types(self):
        self.member_types = {i: 'road' if i.find('road') > -1 else 'beam' for i in self.members}

    def get_member_lengths(self):
        def rel_f(l):
            if l < 1.847:
                return 1. - 0.14633 * l**2
            else:
                return 1.7094 / l**2
        self.member_lengths = {i: ((self._node_position_x[self._node_labels.index(self.members[i][0])] -
                                    self._node_position_x[self._node_labels.index(self.members[i][1])]) ** 2 +
                                   (self._node_position_y[self._node_labels.index(self.members[i][0])] -
                                    self._node_position_y[self._node_labels.index(self.members[i][1])]) ** 2
                                   ) ** 0.5 for i in self.members}
        self.compressive_weaknesses = {i[0]: rel_f(i[1]) for i in self.member_lengths.items()}
        # wood: E = 6.9 GPa, density = 0.7 g/cc, sigy = 34.5 MPa
        # calculate critical length
        # kg m s N Pa J
        # calculate critical load(length)
        # mass / m (if aluminum, 5cm by 5cm square rod, = 6.75 kg / m ??? (5 cm ~ 2 in
        # density = 2.7 g / cm**3  (
        #  A = 25cm**2 (2.5e-3 m**2)
        # I = 52.08 cm**4 = 5.21e-7 m**4
        # R = (I/A)**0.5 = 1.44 cm 1.44e-2 m
        # E = 69 GPa (6.9e10 Pa)
        # Sigma_yield = 83 MPa (8.3e7 Pa)
        # Scrit = 128
        # L crit = 1.84704 m
        # Tensile Failure: 8.3e7*2.5e-3 = 207.5 kN
        # Johnson's Formula (short columns): F = A*sig* [1-(sig/E)*(L/(2*pi*R))**2]
        # F = 207.5 kN * [ 1 - 0.14633 * L**2]
        # Euler's Formula (above L = 1.847 m) F = pi**2 * E * I/ L**2
        # F = 6.9e10 * pi**2 * 5.208e-7 /L**2
        # F = 1.7094 * 207.5 kN / L**2
        # self.member_buckling_loads = {i[0]: i[1]}

    def get_member_masses(self, masses):
        self.member_masses = {i: self.member_lengths[i] * masses[self.member_types[i]] for i in self.members}

    def get_node_masses(self, masses):
        self.node_masses = {i[0]: masses['node'] for i in self.nodes}

    def update_history(self, time):
        self._history['time'].append(time)
        ts = pd.Series(self._internal_forces, dtype=float)
        mt = ts.max()
        mtid = ts.idxmax()
        cs = pd.Series({i: self._internal_forces[i]/self.compressive_weaknesses[i] for i in self._internal_forces}, dtype=float)
        mc = cs.min()
        mcid = cs.idxmin()
        self._history['max_comp'].append(-mc)
        self._history['max_tens'].append(mt)
        self._history['max_tens_id'].append(mtid)
        self._history['max_comp_id'].append(mcid)

    def plotter(self, time=0, name='', group=''):
        print('plotting time =', time)
        scale = 10000
        cdict = {'red': [(0.0, 0.0, 0.0),
                         (0.5, 0.0, 0.0),
                         (1.0, 1.0, 1.0)],

                 'green': [(0.0, 0.0, 0.0),
                           (1.0, 0.0, 0.0)],

                 'blue': [(0.0, 1.0, 1.0),
                          (0.5, 0.0, 0.0),
                          (1.0, 0.0, 0.0)]}
        bx = [[float(self._node_position_x[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_x[self._node_labels.index(self.members[i][1])])]
              for i in self.members]
        rbx = [[float(self._node_position_x[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_x[self._node_labels.index(self.members[i][1])])]
              for i in self.members if i.find('road') > -1]
        xmin = np.min(rbx)
        xmax = np.max(rbx)
        w = xmax-xmin
        by = [[float(self._node_position_y[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_y[self._node_labels.index(self.members[i][1])])]
              for i in self.members]
        rx = [float(i[1]) for i in self.nodes if i[0].find('road') > -1]
        ry = [float(i[2]) for i in self.nodes if i[0].find('road') > -1]
        ymin, ymax = [f(by) for f in [np.min, np.max]]
        dn = mcm.TwoSlopeNorm(vcenter=0, vmin=-1*self._sf,
                              vmax=self._sf)
        cmap = mcm.LinearSegmentedColormap('stress', cdict)

        colors = [cmap(dn(float(i[1]))) if float(i[1]) > 0 else
                  cmap(dn(float(i[1])/float(self.compressive_weaknesses[i[0]])))
                  for i in self.internal_forces.items()]
        labs = ['%.0f\n%s' % (self.internal_forces[i], i) for i in self.members]
        fig, ax = plt.subplots(2, 1, figsize=[12, 12])
        plt.sca(ax[0])
        xposhist = np.multiply(self._history['time'], w)
        plt.plot(xposhist, self._history['max_comp'], 'b-o', label='Compression')
        for xi, yi, ti in zip(xposhist, self._history['max_comp'], self._history['max_comp_id']):
            plt.annotate('<--'+ti, (xi, yi), rotation=70, fontsize='small')
        plt.plot(xposhist, self._history['max_tens'], 'r-o', label='Tension')
        for xi, yi, ti in zip(xposhist, self._history['max_tens'], self._history['max_tens_id']):
            plt.annotate('<--'+ti, (xi, yi), rotation=70, fontsize='small')
        plt.plot([0, w], [self._sf, self._sf], 'k:', label='Load Limit')
        plt.xlabel('X position of unicycle')
        plt.xlim((0, w))
        plt.ylim((0, 2e4))
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        maxf = max([max(self._history['max_comp']), max(self._history['max_tens'])]) / self._sf
        textstr = "Maximum Beam Load Reached %.4f" % maxf + r' $\times$ design limit'
        # place a text box in upper left in axes coords
        ax[0].text(0.05, 0.95, textstr, transform=ax[0].transAxes, fontsize=14,
                verticalalignment='top', bbox=props)
        if time > 0:
            labelLines(plt.gca().get_lines(), xvals=[time*i for i in [0.2, 0.6, 0.5/time]])
        plt.sca(ax[1])
        [plt.plot(x, y, '-', c=c, label=l) for x, y, c, l in zip(bx, by, colors, labs)]
        massx = float(time * xmax)
        f = interp1d(rx, ry)
        massy = f(massx)
        labelLines(plt.gca().get_lines(), xvals=[np.mean(bi) for bi in bx])
        patch = plot_custom_icon(massx, massy)
        ax[1].add_patch(patch)
        plt.plot(np.array(bx).T, np.array(by).T, 'ok')

        for i in self.reaction_loads:
            if self.reaction_loads[i] == 0:
                continue
            ni = self._node_labels.index(i[2:-2])
            x, y = self._node_positions[ni]
            dir = np.sign(self.reaction_loads[i])
            if i[-2:].find('_x'):
                # horiz
                dx = dir * 0.2
                dy = 0
            elif i[-2:].find('_y'):
                # vert
                dx = 0
                dy = dir * 0.2
            w = self.reaction_loads[i]/scale
            plt.arrow(x, y, dy, dx, width=w,
                      head_width=w*1.5, head_length=w*0.5)
        for i in self.loads:
            for ii in self.loads[i]:
                mags = []
                dirs = []
                if isinstance(ii[0], sympy.core.numbers.Float):
                    mags.append(ii[0])
                    dirs.append(ii[1])
            dx = [0.2 * np.sin(np.radians(float(d))) for d in dirs]
            dy = [0.2 * np.cos(np.radians(float(d))) for d in dirs]
            ww = np.divide(mags, sum(mags))
            dx = np.multiply(dx, ww).sum()
            dy = np.multiply(dy, ww).sum()

            x, y = self._node_positions[self._node_labels.index(i)]
            w = float(np.sum(mags)/scale)
            plt.arrow(float(x), float(y), float(dy), float(dx), width=w,
                          head_width=w * 1.5, head_length=w * 0.5, fc='green')
        if (ymax-ymin)*2 > xmax - xmin:
            buf = (ymax - ymin)*2 - (xmax - xmin)
        else:
            buf = 0.5
        plt.xlim((float(xmin)-buf, float(xmax)+buf))
        plt.ylim((float(ymin)-buf/2, float(ymin) + float(xmax)/2 + buf/2))
        plt.title(name + ' ' + group)
        plt.savefig(os.path.join(ff, '%04i.png' % (time*200)), dpi=120)
        if time == 1.0:
            timecode = time_module.strftime('%H:%M')
            plt.savefig('%s_%s.png' % (name, timecode), dpi=120)
            print('See plot saved in files tab:', '%s_%s.png' % (name, timecode))
            print('###Overall Result for %s:'% name, "Maximum Beam Load Reached %.4fx Design Limit" % maxf)
        plt.close()


def icon_data():
    """scalable vector graphics (.svg) for a unicycle"""
    # Unicycle from OpenClipart library by Andy Fitzsimon
    #   <cc:License rdf:about="http://web.resource.org/cc/PublicDomain">
    #     <cc:permits rdf:resource="http://web.resource.org/cc/Reproduction"/>
    #     <cc:permits rdf:resource="http://web.resource.org/cc/Distribution"/>
    #     <cc:permits rdf:resource="http://web.resource.org/cc/DerivativeWorks"/>
    #   </cc:License>
    unicycle = """M 4055,674 C 4506,672 4952,770 5403,767 5793,765 6178,691 6567,684 6759,682 6966,697 7128,812 7276,916 7330,1116 7294,1288 7264,1408 7170,1499 7069,1563 6899,1669 6703,1724 6509,1767 6174,1837 5832,1862 5491,1870 5490,3205 5491,4541 5491,5876 6377,5908 7251,6224 7947,6774 8675,7341 9210,8152 9440,9047 9706,10065 9573,11181 9070,12106 8631,12924 7916,13591 7068,13969 6113,14401 4995,14453 4002,14118 3157,13837 2410,13278 1895,12552 1337,11773 1058,10798 1122,9842 1172,9003 1482,8183 1998,7521 2519,6845 3249,6331 4064,6073 4430,5955 4813,5892 5197,5877 5197,4540 5197,3204 5197,1867 4785,1849 4368,1809 3974,1679 3796,1616 3612,1535 3491,1383 3354,1214 3375,929 3557,800 3700,697 3883,678 4055,674 Z
    M 5402,1062 C 4959,1065 4522,973 4080,967 3969,971 3850,968 3750,1024 3679,1061 3682,1167 3737,1218 3836,1318 3973,1366 4103,1411 4447,1518 4808,1554 5166,1570 5621,1585 6082,1568 6527,1463 6674,1423 6829,1382 6953,1288 7049,1227 7021,1065 6918,1026 6789,970 6643,975 6506,979 6137,997 5771,1059 5402,1062 Z
    M 1682,8664 C 1312,9592 1318,10661 1697,11585 2029,12407 2649,13108 3423,13541 4296,14035 5362,14172 6332,13916 7174,13700 7936,13192 8464,12501 9084,11704 9373,10662 9254,9659 9156,8772 8741,7924 8102,7301 7413,6616 6463,6206 5492,6171 5490,6365 5492,6558 5491,6752 5899,6773 6304,6862 6678,7027 7429,7351 8052,7961 8393,8706 8822,9624 8798,10738 8329,11637 7902,12473 7110,13114 6201,13350 5291,13595 4282,13431 3499,12907 2708,12389 2153,11524 2023,10586 1877,9617 2187,8592 2847,7867 3439,7200 4306,6789 5197,6752 5197,6558 5198,6365 5196,6171 4538,6197 3886,6389 3321,6730 2583,7173 1996,7863 1682,8664 Z
    M 4498,7163 C 3667,7399 2951,8006 2582,8788 2175,9625 2185,10649 2608,11477 3002,12272 3759,12875 4621,13081 5271,13240 5973,13178 6584,12904 7349,12567 7962,11903 8235,11112 8548,10228 8422,9203 7900,8423 7375,7617 6454,7088 5492,7047 5490,7852 5493,8656 5491,9461 5650,9498 5798,9591 5886,9730 5988,9884 6004,10083 5951,10258 6265,10450 6580,10642 6894,10836 6923,10852 6952,10874 6987,10871 7209,10876 7431,10866 7652,10876 7831,10880 7967,11076 7913,11246 7878,11382 7738,11472 7600,11461 7105,11458 6610,11465 6115,11458 5896,11446 5771,11158 5912,10990 6020,10839 6224,10874 6384,10873 6191,10745 5990,10626 5790,10507 5710,10557 5633,10617 5539,10639 5282,10714 4988,10593 4857,10361 4772,10222 4759,10052 4794,9896 4439,9677 4082,9464 3729,9242 3546,9241 3363,9245 3181,9241 3029,9235 2900,9094 2908,8943 2907,8787 3049,8650 3205,8656 3700,8656 4196,8655 4691,8657 4842,8655 4979,8790 4977,8942 4986,9090 4863,9230 4715,9240 4573,9246 4431,9240 4290,9242 4506,9376 4722,9506 4938,9640 5013,9569 5101,9514 5197,9479 5197,8668 5198,7858 5197,7047 4961,7060 4726,7097 4498,7163 Z"""
    vertices = []
    codes = []
    parts = unicycle.split()
    i = 0
    code_map = {
        'M': Path.MOVETO,
        'C': Path.CURVE4,
        'L': Path.LINETO,
        'Z': Path.CLOSEPOLY,

    }
    while i < len(parts):
        if parts[i] in code_map:
            path_code = code_map[parts[i]]
            repeat_offset = 0
        else:
            path_code = path_code
            repeat_offset = -1
        if parts[i] == 'Z':
            repeat_offset -= 1
        npoints = Path.NUM_VERTICES_FOR_CODE[path_code]
        codes.extend([path_code] * npoints)
        if parts[i] == 'Z':
            vertices.append(vertices[-1])
        else:
            vertices.extend([[*map(float, y.split(','))]
                             for y in parts[i + repeat_offset + 1:][:npoints]])
        i += npoints + 1 + repeat_offset

    vertices = np.array(vertices)

    vertices[:, 1] *= -1  # flip vertically
    vertices[:, 0] -= vertices[np.argmin(vertices[:, 1]), 0]  # set bottom of unicycle at origin
    vertices[:, 1] -= np.min(vertices[:, 1])
    #vertices /= np.max(vertices[:, 1])  # scale to h = 1
    vertices /= 10000
    vertices[:, 0] -= 0.02
    return vertices, codes


def icon_data_alt():
    codes = [1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 79,
             1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 79, 1, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 79, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
             4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 79]
    vvv = np.array([[-0.114 , -0.0689, -0.0243,  0.0208,  0.0598,  0.0983,  0.1372,  0.1564,  0.1771,  0.1933,  0.2081,  0.2135,  0.2099,  0.2069,  0.1975,  0.1874,  0.1704,  0.1508,  0.1314,  0.0979,  0.0637,
         0.0296,  0.0295,  0.0296,  0.0296,  0.1182,  0.2056,  0.2752,  0.348 ,  0.4015,  0.4245,  0.4511,  0.4378,  0.3875,  0.3436,  0.2721,  0.1873,  0.0918, -0.02  , -0.1193, -0.2038, -0.2785,
        -0.33  , -0.3858, -0.4137, -0.4073, -0.4023, -0.3713, -0.3197, -0.2676, -0.1946, -0.1131, -0.0765, -0.0382,  0.0002,  0.0002,  0.0002,  0.0002, -0.041 , -0.0827, -0.1221, -0.1399, -0.1583,
        -0.1704, -0.1841, -0.182 , -0.1638, -0.1495, -0.1312, -0.114 , -0.114 ,  0.0207, -0.0236, -0.0673, -0.1115, -0.1226, -0.1345, -0.1445, -0.1516, -0.1513, -0.1458, -0.1359, -0.1222, -0.1092,
        -0.0748, -0.0387, -0.0029,  0.0426,  0.0887,  0.1332,  0.1479,  0.1634,  0.1758,  0.1854,  0.1826,  0.1723,  0.1594,  0.1448,  0.1311,  0.0942,  0.0576,  0.0207,  0.0207, -0.3513, -0.3883,
        -0.3877, -0.3498, -0.3166, -0.2546, -0.1772, -0.0899,  0.0167,  0.1137,  0.1979,  0.2741,  0.3269,  0.3889,  0.4178,  0.4059,  0.3961,  0.3546,  0.2907,  0.2218,  0.1268,  0.0297,  0.0295,
         0.0297,  0.0296,  0.0704,  0.1109,  0.1483,  0.2234,  0.2857,  0.3198,  0.3627,  0.3603,  0.3134,  0.2707,  0.1915,  0.1006,  0.0096, -0.0913, -0.1696, -0.2487, -0.3042, -0.3172, -0.3318,
        -0.3008, -0.2348, -0.1756, -0.0889,  0.0002,  0.0002,  0.0003,  0.0001, -0.0657, -0.1309, -0.1874, -0.2612, -0.3199, -0.3513, -0.3513, -0.0697, -0.1528, -0.2244, -0.2613, -0.302 , -0.301 ,
        -0.2587, -0.2193, -0.1436, -0.0574,  0.0076,  0.0778,  0.1389,  0.2154,  0.2767,  0.304 ,  0.3353,  0.3227,  0.2705,  0.218 ,  0.1259,  0.0297,  0.0295,  0.0298,  0.0296,  0.0455,  0.0603,
         0.0691,  0.0793,  0.0809,  0.0756,  0.107 ,  0.1385,  0.1699,  0.1728,  0.1757,  0.1792,  0.2014,  0.2236,  0.2457,  0.2636,  0.2772,  0.2718,  0.2683,  0.2543,  0.2405,  0.191 ,  0.1415,
         0.092 ,  0.0701,  0.0576,  0.0717,  0.0825,  0.1029,  0.1189,  0.0996,  0.0795,  0.0595,  0.0515,  0.0438,  0.0344,  0.0087, -0.0207, -0.0338, -0.0423, -0.0436, -0.0401, -0.0756, -0.1113,
        -0.1466, -0.1649, -0.1832, -0.2014, -0.2166, -0.2295, -0.2287, -0.2288, -0.2146, -0.199 , -0.1495, -0.0999, -0.0504, -0.0353, -0.0216, -0.0218, -0.0209, -0.0332, -0.048 , -0.0622, -0.0764,
        -0.0905, -0.0689, -0.0473, -0.0257, -0.0182, -0.0094,  0.0002,  0.0002,  0.0003,  0.0002, -0.0234, -0.0469, -0.0697, -0.0697],
       [ 1.3779,  1.3781,  1.3683,  1.3686,  1.3688,  1.3762,  1.3769,  1.3771,  1.3756,  1.3641,  1.3537,  1.3337,  1.3165,  1.3045,  1.2954,  1.289 ,  1.2784,  1.2729,  1.2686,  1.2616,  1.2591,
         1.2583,  1.1248,  0.9912,  0.8577,  0.8545,  0.8229,  0.7679,  0.7112,  0.6301,  0.5406,  0.4388,  0.3272,  0.2347,  0.1529,  0.0862,  0.0484,  0.0052,  0.    ,  0.0335,  0.0616,  0.1175,
         0.1901,  0.268 ,  0.3655,  0.4611,  0.545 ,  0.627 ,  0.6932,  0.7608,  0.8122,  0.838 ,  0.8498,  0.8561,  0.8576,  0.9913,  1.1249,  1.2586,  1.2604,  1.2644,  1.2774,  1.2837,  1.2918,
         1.307 ,  1.3239,  1.3524,  1.3653,  1.3756,  1.3775,  1.3779,  1.3779,  1.3391,  1.3388,  1.348 ,  1.3486,  1.3482,  1.3485,  1.3429,  1.3392,  1.3286,  1.3235,  1.3135,  1.3087,  1.3042,
         1.2935,  1.2899,  1.2883,  1.2868,  1.2885,  1.299 ,  1.303 ,  1.3071,  1.3165,  1.3226,  1.3388,  1.3427,  1.3483,  1.3478,  1.3474,  1.3456,  1.3394,  1.3391,  1.3391,  0.5789,  0.4861,
         0.3792,  0.2868,  0.2046,  0.1345,  0.0912,  0.0418,  0.0281,  0.0537,  0.0753,  0.1261,  0.1952,  0.2749,  0.3791,  0.4794,  0.5681,  0.6529,  0.7152,  0.7837,  0.8247,  0.8282,  0.8088,
         0.7895,  0.7701,  0.768 ,  0.7591,  0.7426,  0.7102,  0.6492,  0.5747,  0.4829,  0.3715,  0.2816,  0.198 ,  0.1339,  0.1103,  0.0858,  0.1022,  0.1546,  0.2064,  0.2929,  0.3867,  0.4836,
         0.5861,  0.6586,  0.7253,  0.7664,  0.7701,  0.7895,  0.8088,  0.8282,  0.8256,  0.8064,  0.7723,  0.728 ,  0.659 ,  0.5789,  0.5789,  0.729 ,  0.7054,  0.6447,  0.5665,  0.4828,  0.3804,
         0.2976,  0.2181,  0.1578,  0.1372,  0.1213,  0.1275,  0.1549,  0.1886,  0.255 ,  0.3341,  0.4225,  0.525 ,  0.603 ,  0.6836,  0.7365,  0.7406,  0.6601,  0.5797,  0.4992,  0.4955,  0.4862,
         0.4723,  0.4569,  0.437 ,  0.4195,  0.4003,  0.3811,  0.3617,  0.3601,  0.3579,  0.3582,  0.3577,  0.3587,  0.3577,  0.3573,  0.3377,  0.3207,  0.3071,  0.2981,  0.2992,  0.2995,  0.2988,
         0.2995,  0.3007,  0.3295,  0.3463,  0.3614,  0.3579,  0.358 ,  0.3708,  0.3827,  0.3946,  0.3896,  0.3836,  0.3814,  0.3739,  0.386 ,  0.4092,  0.4231,  0.4401,  0.4557,  0.4776,  0.4989,
         0.5211,  0.5212,  0.5208,  0.5212,  0.5218,  0.5359,  0.551 ,  0.5666,  0.5803,  0.5797,  0.5797,  0.5798,  0.5796,  0.5798,  0.5663,  0.5511,  0.5363,  0.5223,  0.5213,  0.5207,  0.5213,
         0.5211,  0.5077,  0.4947,  0.4813,  0.4884,  0.4939,  0.4974,  0.5785,  0.6595,  0.7406,  0.7393,  0.7356,  0.729 ,  0.729 ]])
    vertices = vvv.T
    return vertices, codes


def plot_custom_icon(x, y):
    """Showing off some matplotlib features to make a bridge-crossing figure"""
    vertices, codes = icon_data_alt()
    vertices[:, 0] += x
    vertices[:, 1] += y
    uni_path = Path(vertices, codes)
    uni_patch = PathPatch(uni_path, facecolor=(0.2, 0.2, 0.2),
                          edgecolor=(0.0, 0.0, 0.0))
    return uni_patch
    #plt.scatter(x, y, marker=r'$\Phi$', c='k', s=70)


def set_ligament_masses():
    """These change from semester to semester"""
    return {'road': 15,  # kg / m
            'beam': 8,  # kg / m
            'node': 2}  # kg

def get_simulation_timestep_plan(truss):
    rnodes = len([i for i in truss._node_labels if i.find('road') > -1])
    tsteps = np.linspace(0, 1, 2 * rnodes - 1)


def clean_frames():
    ll = os.listdir(ff)
    for li in ll:
        os.remove(os.path.join(ff,li))
    print('cleaned out the directory ./frames/')


def apply_self_load(truss, masses):
    g = 9.81 #(m/s**2)
    truss.get_member_lengths()
    truss.get_member_types()
    truss.get_member_masses(masses)
    truss.get_node_masses(masses)
    efm = {i[0]: truss.node_masses[i[0]] for i in truss.nodes}
    for member in truss.members:
        a, b = truss.members[member]
        efm[a] += truss.member_masses[member] / 2
        efm[b] += truss.member_masses[member] / 2
    for node in truss._node_labels:
        truss.apply_load(node, magnitude=efm[node]*g, direction=270)
    return


def lever_rule(fun, road, pos, v):
    g = 9.81
    keylist = list(road.keys())
    dist = [(i, abs(road[i][1]-pos)) for i in keylist]

    dl2 = [i[1] for i in dist]
    if 0 in dl2:
        nodes = [i[0] for i in dist if i[1] == 0]
        ws = 1
        load = g * v
        fun(road[keylist[nodes[0]]][0], load * ws, 270)
        return
    else:
        dl2.sort()
        lim = max(dl2[0:2])
        nodes = {i[0]: i[1] for i in dist if i[1] <= lim}
        nl = [(i[0], i[1]) for i in nodes.items()]
        ws = {nl[i][0]: nl[-(i+1)][1]/sum(dl2[0:2]) for i in range(len(nodes))}
        # w0 = d1/(d0+d1)
        # w1 = d0/(d0+d1)
        load = g * v

        for i in ws:
            fun(road[keylist[i]][0], load * ws[i], 270)
    return


def apply_moving_loads(truss, t, v, tprev=None):
    road_pins = [i for i in truss._node_labels if i.find('road') > -1]
    road_pin_numbers = [int(i.split('_')[-1]) for i in road_pins]
    road_nodes = {i[0]: truss.nodes[truss._node_labels.index(i[1])] for i in zip(road_pin_numbers, road_pins)}
    road_xvals = [i[1] for i in road_nodes.values()]
    load_pos = t * (max(road_xvals) - min(road_xvals))
    if tprev is None:
        pass
    else:
        prev_pos = tprev * (max(road_xvals) - min(road_xvals))
        lever_rule(truss.remove_load, road_nodes, prev_pos, v)
    lever_rule(truss.apply_load, road_nodes, load_pos, v)
    return


def save_truss(tt, name=None, group=None):
    if name is None:
        name = 'truss' + input("What # round is this? ")
    if group is None:
        group = input("What is your group name? ")
    timecode = time_module.strftime('_%H:%M')
    h = open(name + group + timecode+ '.pickle', 'wb')
    pickle.dump(tt, h)
    h.close()


def smart_pfind(path):
    ll = os.listdir(path)
    ll2 = [li for li in ll if li.find('.pickle') > -1]
    return ll2[int(input('Pick by number:\n' +
                         '\n'.join(['%i: %s' % (i, li) for i, li in enumerate(ll2)])+'\n'))]


def get_truss(name=None):
    if name is None:
        name = smart_pfind('.')
    h = open(name, 'rb')
    tt = pickle.load(h)
    h.close()
    return tt


def l_simplify(loadvec, phi=270):
    return np.sum([i[0] for i in loadvec if isinstance(i[0], sympy.core.numbers.Float) and i[1]==phi])


def audit_loads(truss, t, dataframe):
    if dataframe is None:
        dataframe = {k: [l_simplify(truss.loads[k])] for k in truss.loads}
        dataframe['index'] = [t]
    else:
        for k in truss.loads:
            dataframe[k].append(l_simplify(truss.loads[k]))
        dataframe['index'].append(t)
    return dataframe


def plot_full_history(ldf, ifdf):
    ldf2 = pd.DataFrame(ldf).set_index('index').astype(float)
    ldf2.plot()
    plt.title("Load history per node")
    plt.savefig('plotting_loading_history_most_recent.png')
    plt.close()
    # for internal forces, only plot most relevant beams (top 3 compressive, top 3 tensile)
    ifdf2 = pd.DataFrame(ifdf).set_index('index').astype(float)
    comp = ifdf2.min(axis=0).nsmallest(3)
    tens = ifdf2.max(axis=0).nlargest(3)
    ifdf3 = ifdf2.loc[:, list(tens.index)+list(comp.index)]
    ifdf3.plot()
    plt.title("Internal Forces per beam")
    plt.savefig('plotting_internal_history_most_recent.png')
    plt.close()
    # plot the compressive bucking vs length:
    def rel_f(l):
        if l < 1.847:
            return 1. - 0.14633 * l ** 2
        else:
            return 1.7094 / l ** 2
    x = np.linspace(0.1, 6)
    plt.plot(x, np.array([rel_f(xi) for xi in x]))
    plt.ylabel('relative compressive beam strength')
    plt.xlabel('Beam length')
    plt.savefig('Compressive_strength_guide.png')
    plt.close()


def audit_internal_forces(truss, t, dataframe=None):
    if dataframe is None:
        dataframe = {k: [truss.internal_forces[k]] for k in truss.internal_forces}
        dataframe['index'] = [t]
    else:
        for k in truss.internal_forces:
            dataframe[k].append(truss.internal_forces[k])
        dataframe['index'].append(t)
    return dataframe


def check_truss_roads(tt):
    """Verify that the truss includes enough road sections to cross"""
    road_joints = sum([1 for i in tt.nodes if i[0].find('road')> -1])
    road_sections = sum([1 for i in tt.members if i.find('road')>-1])
    assert road_sections >= 4, "To be considered valid for this project, the bridge must have at least 4 road sections"
    assert road_sections == road_joints - 1, "Wrong number of road sections and/or nodes"
    sections_on = []
    for node in tt.nodes:
        connected = 0
        if node[0].find('road')>-1:
            for key in tt.members:
                if key.find('road')>-1 and node[0] in tt.members[key]:
                    connected += 1
            sections_on.append((node[0], connected))
    test_on = [i[1] for i in sections_on]
    assert min(test_on) == 1, sections_on
    assert max(test_on) == 2, sections_on
    assert np.count_nonzero(np.array(test_on) == 1) == 2, 'Only the endpoints should have a single road section connected'


def check_truss_origin(tt):
    for i in tt.nodes:
        if i[0] == 'road_joint_0':
            if i[1] == 0 and i[2] == 0:
                print('road_joint_0 found at origin, as required')
                return
            else:
                print('road_joint_0 found, but x,y not 0,0!', *i)
                a = input('Do you want to continue with this invalid design? (y/N)')
                return
    print('road_joint_0 not found. Did you change the naming scheme? Your origin is unverified!')

In [ ]:
# @title Plan your designing using functions, with these examples as templates (copy me and rename to experiment)


def make_bridge(length=8, segments=4, height=3):
    """This example shows how to add nodes and members (a.k.a. joints and beams)

    Note that you can add any arguments you need and supply default values using kwargs"""
    tt = Truss2()
    # build road
    # add nodes at regular intervals between x = 0 and x = length
    for i in range(segments + 1):
        tt.add_node('road_joint_%i' % i, i * length/segments, 0)
    # add nodes at y == height and at x values between the road nodes
    for i in range(1, segments + 1):
        tt.add_node('truss_node_%i' % i, (i - 0.5) * length/segments, height)
        # make beams to connect to the road nodes
        tt.add_member('beam_%ia' % i, 'road_joint_%i' % (i-1), 'truss_node_%i' % i)
        tt.add_member('beam_%ib' % i, 'road_joint_%i' % i, 'truss_node_%i' % i)

    for i in range(2, segments + 1):
        tt.add_member('beam_%ic' % i, 'truss_node_%i' % (i - 1), 'truss_node_%i' % i)
    for i in range(1, segments + 1):
        tt.add_member('road_section_%i' % i, 'road_joint_%i' % (i-1), 'road_joint_%i' % i)
    tt.apply_support('road_joint_0', 'pinned')
    tt.apply_support('road_joint_%i' % segments, 'roller')
    return tt


def make_f_bridge(length=8, segments=4, f_y_road=0, f_y_pins=3, f_x_pins=0):
    """This is an approach to defining node positions by function.

    This function will always place one less horizontal support beam than road segments.
    You can make much better bridges if you discover other valid truss designs...
    Parameters:
        length: float or int
        segments: int
        f_y_road: int, float, or Callable
        f_y_pins: int, float, or Callable
        f_x_pins: int, float, or Callable

        Any Callable passed to the parameters above needs to have the signature:
         f(x, xmax) -> float or int
         where x is the current x position along the bridge (assuming equal spacing)
         and xmax is the length of the bridge."""
    tt = Truss2()
    if isinstance(f_y_road, (int, float)):
        val = f_y_road

        def f_y_road(x, xmax):
            return val

    if isinstance(f_y_pins, (int, float)):
        val = f_y_pins

        def f_y_pins(x, xmax):
            return val

    if isinstance(f_x_pins, (int, float)):
        val = f_x_pins
        def f_x_pins(x, xmax):
            return val + x

    # build road
    for i in range(segments + 1):
        tt.add_node('road_joint_%i' % i, i * length/segments, f_y_road(i * length/segments, length))
    for i in range(1, segments + 1):
        xp = f_x_pins(i * length/segments, length)
        tt.add_node('truss_node_%i' % i, xp, f_y_pins(xp, length))
        tt.add_member('beam_%ia' % i, 'road_joint_%i' % (i-1), 'truss_node_%i' % i)
        tt.add_member('beam_%ib' % i, 'road_joint_%i' % i, 'truss_node_%i' % i)
    for i in range(2, segments + 1):
        tt.add_member('beam_%ic' % i, 'truss_node_%i' % (i - 1), 'truss_node_%i' % i)
    for i in range(1, segments + 1):
        tt.add_member('road_section_%i' % i, 'road_joint_%i' % (i-1), 'road_joint_%i' % i)
    tt.apply_support('road_joint_0', 'pinned')
    tt.apply_support('road_joint_%i' % segments, 'roller')
    return tt

## copy one of the above below here and rename it, then try out changes!

In [ ]:
# @title Use the class Simulator or the function truss_simulator to test your bridge
class Simulator:

    def __init__(self, name=None, group=None, moving_mass=50):
        if name is None:
            self.name = 'truss' + input("What is your name? ")
        else:
            self.name = name
        if group is None:
            self.group = input("What is your group name? ")
        else:
            self.group = group
        self.moving_mass = moving_mass
        self.ligament_masses = set_ligament_masses()

    def run(self, truss, tsteps=None, save=True, verbose=True):
        if tsteps is None:
            rnodes = len([i for i in truss._node_labels if i.find('road') > -1])
            tsteps = np.linspace(0, 1, 2 * rnodes - 1)
        check_truss_origin(truss)
        check_truss_roads(truss)
        apply_self_load(truss, self.ligament_masses)
        apply_moving_loads(truss, tsteps[0], self.moving_mass)
        ldf = audit_loads(truss, tsteps[0], None)
        truss.solve()
        ifdf = audit_internal_forces(truss, tsteps[0], None)
        truss.update_history(tsteps[0])
        for ii, t in enumerate(tsteps[1:]):
            oldloads = copy.deepcopy(truss.loads)
            apply_moving_loads(truss, t, self.moving_mass, tsteps[ii])
            ldf = audit_loads(truss, t, ldf)
            if verbose:
                changes = ['%s load: %s -> %s' % (i, oldloads[i][-1][0], truss.loads[i][-1][0]) for i in truss.loads if
                           oldloads[i] != truss.loads[i]]
                print(ii + 1, t, tsteps[ii])
                print(*changes, sep='\n')
            truss.solve()
            ifdf = audit_internal_forces(truss, t, ifdf)
            truss.update_history(t)
            if t == 1.0:
                truss.plotter(t, name=self.name, group=self.group)
        if save:
            save_truss(truss, self.name, self.group)
        plot_full_history(ldf, ifdf)
        maxforce = max([max(truss._history['max_comp']), max(truss._history['max_tens'])]) / truss._sf
        return maxforce, ldf, ifdf


def truss_simulator(truss, moving_mass=50, name=None, group=None, tsteps=None, save=True, verbose=True):
    """Different Rounds may have different lengths, number of segments, or arbitrary requirements on shape (truss below road,
    truss above road, """
    if name is None:
        name = 'truss' + input("What is your name? ")
    if group is None:
        group = input("What is your group name? ")
    ligament_masses = set_ligament_masses()
    if tsteps is None:
        rnodes = len([i for i in truss._node_labels if i.find('road') > -1])
        tsteps = np.linspace(0, 1, 2*rnodes-1)
    check_truss_origin(truss)
    check_truss_roads(truss)
    apply_self_load(truss, ligament_masses)
    apply_moving_loads(truss, tsteps[0], moving_mass)
    ldf = audit_loads(truss, tsteps[0], None)
    truss.solve()
    ifdf = audit_internal_forces(truss, tsteps[0], None)
    truss.update_history(tsteps[0])
    for ii, t in enumerate(tsteps[1:]):
        oldloads = copy.deepcopy(truss.loads)
        apply_moving_loads(truss, t, moving_mass, tsteps[ii])
        ldf = audit_loads(truss, t, ldf)
        if verbose:
            changes = ['%s load: %s -> %s' % (i, oldloads[i][-1][0], truss.loads[i][-1][0]) for i in truss.loads if oldloads[i] != truss.loads[i]]
            print(ii + 1, t, tsteps[ii])
            print(*changes, sep='\n')
        truss.solve()
        ifdf = audit_internal_forces(truss, t, ifdf)
        truss.update_history(t)
        if t == 1.0:
            truss.plotter(t, name=name, group=group)
    if save:
        save_truss(truss, name, group)
    plot_full_history(ldf, ifdf)
    maxforce = max([max(truss._history['max_comp']), max(truss._history['max_tens'])]) / truss._sf
    return maxforce, ldf, ifdf

In [ ]:
# @title Use this cell and below to plan out your interface, test your designers, etc.
tt = make_bridge(length=10) # what arguments do I take?
s = Simulator(name='testing', group='sad_group')
data_tuple = s.run(tt) # what arguments do I take?

road_joint_0 found at origin, as required
1 0.125 0.0
road_joint_0 load: 490.500000000000 -> 245.250000000000
road_joint_1 load: 642.555000000000 -> 245.250000000000
2 0.25 0.125
road_joint_0 load: 245.250000000000 -> 331.087500000000
road_joint_1 load: 245.250000000000 -> 490.500000000000
3 0.375 0.25
road_joint_1 load: 490.500000000000 -> 245.250000000000
road_joint_2 load: 642.555000000000 -> 245.250000000000
4 0.5 0.375
road_joint_1 load: 245.250000000000 -> 642.555000000000
road_joint_2 load: 245.250000000000 -> 490.500000000000
5 0.625 0.5
road_joint_2 load: 490.500000000000 -> 245.250000000000
road_joint_3 load: 642.555000000000 -> 245.250000000000
6 0.75 0.625
road_joint_2 load: 245.250000000000 -> 642.555000000000
road_joint_3 load: 245.250000000000 -> 490.500000000000
7 0.875 0.75
road_joint_4 load: 331.087500000000 -> 245.250000000000
road_joint_3 load: 490.500000000000 -> 245.250000000000
8 1.0 0.875
road_joint_4 load: 245.250000000000 -> 490.500000000000
road_joint_3 load:

In [ ]:
tt._history

{'time': [0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0],
 'max_comp': [12099.338652268341,
  13330.614634520884,
  14561.89061677343,
  14151.465289355914,
  13741.039961938402,
  14151.465289355916,
  14561.890616773431,
  13330.614634520887,
  12099.338652268341],
 'max_tens': [1680.9843750000002,
  1808.71875,
  1952.8031250000001,
  1962.0,
  1987.5468750000002,
  1962.0,
  1952.8031250000006,
  1808.71875,
  1680.9843750000002],
 'max_comp_id': ['beam_4b',
  'beam_1a',
  'beam_1a',
  'beam_1a',
  'beam_4b',
  'beam_4b',
  'beam_4b',
  'beam_4b',
  'beam_4b'],
 'max_tens_id': ['road_section_2',
  'road_section_2',
  'beam_1b',
  'road_section_2',
  'road_section_2',
  'road_section_3',
  'beam_4a',
  'road_section_3',
  'road_section_2']}

In [ ]:
# @title Use this as a scratch cell to try out things and learn

new_truss = Truss2()
help(new_truss.add_node)

Help on method add_node in module sympy.physics.continuum_mechanics.truss:

add_node(label, x, y) method of __main__.Truss2 instance
    This method adds a node to the truss along with its name/label and its location.
    
    Parameters
    label:  String or a Symbol
        The label for a node. It is the only way to identify a particular node.
    
    x: Sympifyable
        The x-coordinate of the position of the node.
    
    y: Sympifyable
        The y-coordinate of the position of the node.
    
    Examples
    
    >>> from sympy.physics.continuum_mechanics.truss import Truss
    >>> t = Truss()
    >>> t.add_node('A', 0, 0)
    >>> t.nodes
    [('A', 0, 0)]
    >>> t.add_node('B', 3, 0)
    >>> t.nodes
    [('A', 0, 0), ('B', 3, 0)]



In [1]:
# @title Custom Bridge Design Function

def custom_bridge_design(length=8.4, segments=4, max_height=8.1, min_clear=0.3, exit_height=0.0,
                         f_y_road=0, f_y_pins=3, f_x_pins=0):
    """Design a truss bridge with specified parameters.

    Parameters:
        length (float): Length of the bridge.
        segments (int): Number of segments in the bridge.
        max_height (float): Maximum height of the truss.
        min_clear (float): Minimum clearance of the bridge.
        exit_height (float): Height at the exit point.
        f_y_road (int, float, or Callable): Function for y-coordinate of road nodes.
        f_y_pins (int, float, or Callable): Function for y-coordinate of truss nodes.
        f_x_pins (int, float, or Callable): Function for x-coordinate of truss nodes.

    Returns:
        Truss2: Instance of Truss2 class representing the designed bridge.
    """
    # Ensure the design limit is between 1-2
    design_limit = max_height / min_clear
    score_to_beat = 1.0  # Set your desired design limit here

    if not (1 <= design_limit <= 2):
        raise ValueError(f"Design limit violation. Design limit: {design_limit}, Score to beat: {score_to_beat}")

    tt = Truss2()
    if callable(f_y_road):
        def y_road(x, xmax):
            return f_y_road(x, xmax)
    else:
        y_road = lambda x, xmax: f_y_road

    if callable(f_y_pins):
        def y_pins(x, xmax):
            return f_y_pins(x, xmax)
    else:
        y_pins = lambda x, xmax: f_y_pins

    if callable(f_x_pins):
        def x_pins(x, xmax):
            return f_x_pins(x, xmax) + x
    else:
        x_pins = lambda x, xmax: f_x_pins + x

    # Build road
    for i in range(segments + 1):
        tt.add_node('road_joint_%i' % i, i * length/segments, y_road(i * length/segments, length))
    for i in range(1, segments + 1):
        xp = x_pins(i * length/segments, length)
        tt.add_node('truss_node_%i' % i, xp, y_pins(xp, length))
        tt.add_member('beam_%ia' % i, 'road_joint_%i' % (i-1), 'truss_node_%i' % i)
        tt.add_member('beam_%ib' % i, 'road_joint_%i' % i, 'truss_node_%i' % i)
    for i in range(2, segments + 1):
        tt.add_member('beam_%ic' % i, 'truss_node_%i' % (i - 1), 'truss_node_%i' % i)
    for i in range(1, segments + 1):
        tt.add_member('road_section_%i' % i, 'road_joint_%i' % (i-1), 'road_joint_%i' % i)
    tt.apply_support('road_joint_0', 'pinned')
    tt.apply_support('road_joint_%i' % segments, 'roller')

    return tt


In [ ]:
# @title Custom Bridge Design and Analysis

import pickle
import numpy as np
from random import uniform
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib import cm as mcm
from sympy.physics.continuum_mechanics.truss import Truss

# Paste the Truss2 class code here
class Truss2(Truss):
    def __init__(self):
        """
        Initializes the class
        """
        self._nodes = []
        self._members = {}
        self._loads = {}
        self._supports = {}
        self._node_labels = []
        self._node_positions = []
        self._node_position_x = []
        self._node_position_y = []
        self._nodes_occupied = {}
        self._reaction_loads = {}
        self._internal_forces = {}
        self._node_coordinates = {}
        self._history = {'time': [], 'max_comp': [], 'max_tens': [],'max_comp_id': [], 'max_tens_id': []}
        self._sf = 3e3

    def get_member_types(self):
        self.member_types = {i: 'road' if i.find('road') > -1 else 'beam' for i in self.members}

    def get_member_lengths(self):
        def rel_f(l):
            if l < 1.847:
                return 1. - 0.14633 * l**2
            else:
                return 1.7094 / l**2
        self.member_lengths = {i: ((self._node_position_x[self._node_labels.index(self.members[i][0])] -
                                    self._node_position_x[self._node_labels.index(self.members[i][1])]) ** 2 +
                                   (self._node_position_y[self._node_labels.index(self.members[i][0])] -
                                    self._node_position_y[self._node_labels.index(self.members[i][1])]) ** 2
                                   ) ** 0.5 for i in self.members}
        self.compressive_weaknesses = {i[0]: rel_f(i[1]) for i in self.member_lengths.items()}
        # wood: E = 6.9 GPa, density = 0.7 g/cc, sigy = 34.5 MPa
        # calculate critical length
        # kg m s N Pa J
        # calculate critical load(length)
        # mass / m (if aluminum, 5cm by 5cm square rod, = 6.75 kg / m ??? (5 cm ~ 2 in
        # density = 2.7 g / cm**3  (
        #  A = 25cm**2 (2.5e-3 m**2)
        # I = 52.08 cm**4 = 5.21e-7 m**4
        # R = (I/A)**0.5 = 1.44 cm 1.44e-2 m
        # E = 69 GPa (6.9e10 Pa)
        # Sigma_yield = 83 MPa (8.3e7 Pa)
        # Scrit = 128
        # L crit = 1.84704 m
        # Tensile Failure: 8.3e7*2.5e-3 = 207.5 kN
        # Johnson's Formula (short columns): F = A*sig* [1-(sig/E)*(L/(2*pi*R))**2]
        # F = 207.5 kN * [ 1 - 0.14633 * L**2]
        # Euler's Formula (above L = 1.847 m) F = pi**2 * E * I/ L**2
        # F = 6.9e10 * pi**2 * 5.208e-7 /L**2
        # F = 1.7094 * 207.5 kN / L**2
        # self.member_buckling_loads = {i[0]: i[1]}

    def get_member_masses(self, masses):
        self.member_masses = {i: self.member_lengths[i] * masses[self.member_types[i]] for i in self.members}

    def get_node_masses(self, masses):
        self.node_masses = {i[0]: masses['node'] for i in self.nodes}

    def update_history(self, time):
        self._history['time'].append(time)
        ts = pd.Series(self._internal_forces, dtype=float)
        mt = ts.max()
        mtid = ts.idxmax()
        cs = pd.Series({i: self._internal_forces[i]/self.compressive_weaknesses[i] for i in self._internal_forces}, dtype=float)
        mc = cs.min()
        mcid = cs.idxmin()
        self._history['max_comp'].append(-mc)
        self._history['max_tens'].append(mt)
        self._history['max_tens_id'].append(mtid)
        self._history['max_comp_id'].append(mcid)

    def plotter(self, time=0, name='', group=''):
        print('plotting time =', time)
        scale = 10000
        cdict = {'red': [(0.0, 0.0, 0.0),
                         (0.5, 0.0, 0.0),
                         (1.0, 1.0, 1.0)],

                 'green': [(0.0, 0.0, 0.0),
                           (1.0, 0.0, 0.0)],

                 'blue': [(0.0, 1.0, 1.0),
                          (0.5, 0.0, 0.0),
                          (1.0, 0.0, 0.0)]}
        bx = [[float(self._node_position_x[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_x[self._node_labels.index(self.members[i][1])])]
              for i in self.members]
        rbx = [[float(self._node_position_x[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_x[self._node_labels.index(self.members[i][1])])]
              for i in self.members if i.find('road') > -1]
        xmin = np.min(rbx)
        xmax = np.max(rbx)
        w = xmax-xmin
        by = [[float(self._node_position_y[self._node_labels.index(self.members[i][0])]),
               float(self._node_position_y[self._node_labels.index(self.members[i][1])])]
              for i in self.members]
        rx = [float(i[1]) for i in self.nodes if i[0].find('road') > -1]
        ry = [float(i[2]) for i in self.nodes if i[0].find('road') > -1]
        ymin, ymax = [f(by) for f in [np.min, np.max]]
        dn = mcm.TwoSlopeNorm(vcenter=0, vmin=-1*self._sf,
                              vmax=self._sf)
        cmap = mcm.LinearSegmentedColormap('stress', cdict)

        colors = [cmap(dn(float(i[1]))) if float(i[1]) > 0 else
                  cmap(dn(float(i[1])/float(self.compressive_weaknesses[i[0]])))
                  for i in self.internal_forces.items()]
        labs = ['%.0f\n%s' % (self.internal_forces[i], i) for i in self.members]
        fig, ax = plt.subplots(2, 1, figsize=[12, 12])
        plt.sca(ax[0])
        xposhist = np.multiply(self._history['time'], w)
        plt.plot(xposhist, self._history['max_comp'], 'b-o', label='Compression')
        for xi, yi, ti in zip(xposhist, self._history['max_comp'], self._history['max_comp_id']):
            plt.annotate('<--'+ti, (xi, yi), rotation=70, fontsize='small')
        plt.plot(xposhist, self._history['max_tens'], 'r-o', label='Tension')
        for xi, yi, ti in zip(xposhist, self._history['max_tens'], self._history['max_tens_id']):
            plt.annotate('<--'+ti, (xi, yi), rotation=70, fontsize='small')
        plt.plot([0, w], [self._sf, self._sf], 'k:', label='Load Limit')
        plt.xlabel('X position of unicycle')
        plt.xlim((0, w))
        plt.ylim((0, 2e4))
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        maxf = max([max(self._history['max_comp']), max(self._history['max_tens'])]) / self._sf
        textstr = "Maximum Beam Load Reached %.4f" % maxf + r' $\times$ design limit'
        # place a text box in upper left in axes coords
        ax[0].text(0.05, 0.95, textstr, transform=ax[0].transAxes, fontsize=14,
                verticalalignment='top', bbox=props)
        if time > 0:
            labelLines(plt.gca().get_lines(), xvals=[time*i for i in [0.2, 0.6, 0.5/time]])
        plt.sca(ax[1])
        [plt.plot(x, y, '-', c=c, label=l) for x, y, c, l in zip(bx, by, colors, labs)]
        massx = float(time * xmax)
        f = interp1d(rx, ry)
        massy = f(massx)
        labelLines(plt.gca().get_lines(), xvals=[np.mean(bi) for bi in bx])
        patch = plot_custom_icon(massx, massy)
        ax[1].add_patch(patch)
        plt.plot(np.array(bx).T, np.array(by).T, 'ok')

        for i in self.reaction_loads:
            if self.reaction_loads[i] == 0:
                continue
            ni = self._node_labels.index(i[2:-2])
            x, y = self._node_positions[ni]
            dir = np.sign(self.reaction_loads[i])
            if i[-2:].find('_x'):
                # horiz
                dx = dir * 0.2
                dy = 0
            elif i[-2:].find('_y'):
                # vert
                dx = 0
                dy = dir * 0.2
            w = self.reaction_loads[i]/scale
            plt.arrow(x, y, dy, dx, width=w,
                      head_width=w*1.5, head_length=w*0.5)
        for i in self.loads:
            for ii in self.loads[i]:
                mags = []
                dirs = []
                if isinstance(ii[0], sympy.core.numbers.Float):
                    mags.append(ii[0])
                    dirs.append(ii[1])
            dx = [0.2 * np.sin(np.radians(float(d))) for d in dirs]
            dy = [0.2 * np.cos(np.radians(float(d))) for d in dirs]
            ww = np.divide(mags, sum(mags))
            dx = np.multiply(dx, ww).sum()
            dy = np.multiply(dy, ww).sum()

            x, y = self._node_positions[self._node_labels.index(i)]
            w = float(np.sum(mags)/scale)
            plt.arrow(float(x), float(y), float(dy), float(dx), width=w,
                          head_width=w * 1.5, head_length=w * 0.5, fc='green')
        if (ymax-ymin)*2 > xmax - xmin:
            buf = (ymax - ymin)*2 - (xmax - xmin)
        else:
            buf = 0.5
        plt.xlim((float(xmin)-buf, float(xmax)+buf))
        plt.ylim((float(ymin)-buf/2, float(ymin) + float(xmax)/2 + buf/2))
        plt.title(name + ' ' + group)
        plt.savefig(os.path.join(ff, '%04i.png' % (time*200)), dpi=120)
        # if time == 1.0:
        #     timecode = time_module.strftime('%H:%M')
        #     plt.savefig('%s_%s.png' % (name, timecode), dpi=120)
        #     print('See plot saved in files tab:', '%s_%s.png' % (name, timecode))
        #     print('###Overall Result for %s:'% name, "Maximum Beam Load Reached %.4fx Design Limit" % maxf)
        # plt.close()

# Continue with the custom_bridge_design function
def custom_bridge_design(length=8.4, segments=4, max_height=8.1, exit_height=0.0, min_clear=0.3, moving_mass=50):
    # Create a Truss2 instance
    tt = Truss2()

    # Build road
    for i in range(segments + 1):
        tt.add_node(f'road_joint_{i}', i * length / segments, 0)

    # Build truss nodes and beams
    for i in range(1, segments + 1):
        xp = uniform(0, length)
        tt.add_node(f'truss_node_{i}', xp, uniform(0, max_height))
        tt.add_member(f'beam_{i}a', f'road_joint_{i - 1}', f'truss_node_{i}')
        tt.add_member(f'beam_{i}b', f'road_joint_{i}', f'truss_node_{i}')

    # Connect truss nodes
    for i in range(2, segments + 1):
        tt.add_member(f'beam_{i}c', f'truss_node_{i - 1}', f'truss_node_{i}')

    # Add road sections
    for i in range(1, segments + 1):
        tt.add_member(f'road_section_{i}', f'road_joint_{i - 1}', f'road_joint_{i}')

    # Apply supports
    tt.apply_support(f'road_joint_0', 'pinned')
    tt.apply_support(f'road_joint_{segments}', 'roller')

    # Save the designed bridge
    with open('bridge_design.pickle', 'wb') as file:
        pickle.dump(tt, file)

    return tt

def interactive_bridge_design():
    # Get user input for bridge design parameters
    length = float(input("Enter the length of the bridge: "))
    segments = int(input("Enter the number of segments: "))
    max_height = float(input("Enter the maximum height of the truss: "))
    exit_height = float(input("Enter the exit height: "))
    min_clear = float(input("Enter the minimum clearance along the bottom: "))
    moving_mass = float(input("Enter the mass of the crossing vehicle: "))

    # Design the bridge based on user input
    custom_bridge_design(length, segments, max_height, exit_height, min_clear, moving_mass)

# Uncomment the line below to run the interactive design
interactive_bridge_design()


In [ ]:
# @title Downloading the actual pickle
from google.colab import files

files.download('/content/bridge_design.pickle')


In [ ]:
# @title Loading the Truss2 Instance from Pickle File
import pickle

# Load the saved Truss2 instance from the pickle file
with open('/content/bridge_design.pickle', 'rb') as file:
    loaded_truss_instance = pickle.load(file)

# Now you can access and explore the contents of the loaded_truss_instance
print(loaded_truss_instance)


In [ ]:
# @title Inspecting Truss2 Instance Attributes
print("Nodes:", loaded_truss_instance.nodes)
print("Members:", loaded_truss_instance.members)
print("Loads:", loaded_truss_instance.loads)
print("Supports:", loaded_truss_instance.supports)
